In [9]:
pip install pymorphy3

In [10]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import math
import pymorphy3

# Скачиваем необходимые ресурсы NLTK
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
nltk.download('punkt_tab')

morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [11]:
stop_words = set(stopwords.words('russian'))



In [12]:
texts = [
    "Летучие мыши увидели кошек с самыми яркими полосками, висящих вниз головой у них за ноги",
    "Кошка сидит с летучими мышами на полосатом коврике под множеством гусей"
]

In [13]:
def preprocess(text):
    stop_words = set(stopwords.words('russian'))
    tokens = nltk.word_tokenize(text.lower())
    return [morph.parse(t)[0].normal_form for t in tokens if t.isalpha() and t not in stop_words]

def bag_of_words_from_processed(processed_texts):
    all_words = []
    for text_tokens in processed_texts:
        all_words.extend(text_tokens)  # Добавляем токены каждого текста в общий список
    return dict(Counter(all_words))


def tf(text):
  words = preprocess(text)
  counts = Counter(words)
  total = len(words)
  return {word: count / total for word, count in counts.items()}

def idf(texts):
    num_docs = len(texts)
    all_words = set()
    for text in texts:
      all_words.update(preprocess(text))

    idfs = {}
    for word in all_words:
        count = sum(1 for text in texts if word in preprocess(text))
        idfs[word] = math.log(num_docs / (count + 1))
    return idfs

def tf_idf(texts):
  idfs = idf(texts)
  tf_idfs = {}
  for i, text in enumerate(texts):
    tf_values = tf(text)
    tf_idfs[str(i)] = {word: tf_values[word] * idfs[word] for word in tf_values}
  return tf_idfs

In [14]:
processed_texts = [preprocess(text) for text in texts]
print(processed_texts)

[['летучий', 'мышь', 'увидеть', 'кошка', 'самый', 'яркий', 'полоска', 'висеть', 'вниз', 'голова', 'нога'], ['кошка', 'сидеть', 'летучий', 'мышь', 'полосатый', 'коврик', 'множество', 'гусь']]


In [15]:
bow = bag_of_words_from_processed(processed_texts)
print("Bag of Words:", bow)

Bag of Words: {'летучий': 2, 'мышь': 2, 'увидеть': 1, 'кошка': 2, 'самый': 1, 'яркий': 1, 'полоска': 1, 'висеть': 1, 'вниз': 1, 'голова': 1, 'нога': 1, 'сидеть': 1, 'полосатый': 1, 'коврик': 1, 'множество': 1, 'гусь': 1}


In [16]:
result_td_idf = tf_idf(bow)
print(result_td_idf)

{'0': {'летучий': 2.0794415416798357}, '1': {'мышь': 2.0794415416798357}, '2': {'увидеть': 2.0794415416798357}, '3': {'кошка': 2.0794415416798357}, '4': {'самый': 2.0794415416798357}, '5': {'яркий': 2.0794415416798357}, '6': {'полоска': 2.0794415416798357}, '7': {'висеть': 2.0794415416798357}, '8': {'вниз': 2.0794415416798357}, '9': {'голова': 2.0794415416798357}, '10': {'нога': 2.0794415416798357}, '11': {'сидеть': 2.0794415416798357}, '12': {'полосатый': 2.0794415416798357}, '13': {'коврик': 2.0794415416798357}, '14': {'множество': 2.0794415416798357}, '15': {'гусь': 2.0794415416798357}}
